In [1]:
# Schnorr ZKP Lab - Interactive and Non-Interactive implementations
import random, hashlib

# Parameters (educational demo)
p = 23
q = 11

# find a generator of subgroup of order q
def find_generator(p, q):
    for h in range(2, p-1):
        g = pow(h, (p-1)//q, p)
        if g != 1:
            return g
g = find_generator(p, q)

# key generation (secret x, public y)
def keygen(q, g, p):
    x = random.randrange(1, q)
    y = pow(g, x, p)
    return x, y

# Schnorr interactive proof
def schnorr_prove_interactive(x, y, g, p, q):
    r = random.randrange(0, q)
    t = pow(g, r, p)
    e = random.randint(0,1)
    s = (r + e * x) % q
    left = pow(g, s, p)
    right = (t * pow(y, e, p)) % p
    passed = (left == right)
    return {'r': r, 't': t, 'e': e, 's': s, 'left': left, 'right': right, 'passed': passed}

# Cheating prover attempt (no secret x)
def cheating_prover_attempt(y, g, p, q):
    s = random.randrange(0, q)
    t = pow(g, s, p)
    e = random.randint(0,1)
    left = pow(g, s, p)
    right = (t * pow(y, e, p)) % p
    passed = (left == right)
    return {'s': s, 't': t, 'e': e, 'left': left, 'right': right, 'passed': passed}

# Fiat-Shamir non-interactive Schnorr
def fiat_shamir_prove(x, y, g, p, q, message=b''):
    r = random.randrange(0, q)
    t = pow(g, r, p)
    h = hashlib.sha256()
    h.update(str(t).encode() + b'|' + message)
    e = int.from_bytes(h.digest(), 'big') % 2
    s = (r + e * x) % q
    left = pow(g, s, p)
    right = (t * pow(y, e, p)) % p
    passed = (left == right)
    return {'r': r, 't': t, 'e': e, 's': s, 'left': left, 'right': right, 'passed': passed}

# Demo run
random.seed(42)
x, y = keygen(q, g, p)
honest = schnorr_prove_interactive(x, y, g, p, q)
cheat_example = cheating_prover_attempt(y, g, p, q)

def cheating_experiment(y, g, p, q, trials=1000):
    success = 0
    for _ in range(trials):
        res = cheating_prover_attempt(y, g, p, q)
        if res['passed']:
            success += 1
    return success / trials, success

cheat_rate, cheat_count = cheating_experiment(y, g, p, q, trials=1000)
message = b"Schnorr test message"
fs = fiat_shamir_prove(x, y, g, p, q, message=message)

# Print results (same content as above)
print("=== Schnorr ZKP (Interactive) - Honest Run ===")
print(f"Parameters: p={p}, q={q}, g={g}")
print(f"Secret x = {x}")
print(f"Public key y = g^x mod p = {y}")
print(f"\nProver commitment t = {honest['t']} (r = {honest['r']})")
print(f"Verifier challenge e = {honest['e']}")
print(f"Response s = {honest['s']}")
print(f"Verification: g^s = {honest['left']}, t * y^e = {honest['right']} -> Passed = {honest['passed']}")

print("\n--- Cheating Prover Example ---")
print(f"Cheating prover prepared t = {cheat_example['t']}, s = {cheat_example['s']}")
print(f"Verifier challenge e = {cheat_example['e']}")
print(f"Verification: g^s = {cheat_example['left']}, t * y^e = {cheat_example['right']} -> Passed = {cheat_example['passed']}")

print("\n--- Cheating Probability Experiment ---")
print(f"Trials = 1000, Successful cheats = {cheat_count}, Empirical success rate = {cheat_rate:.4f}")

print("\n--- Fiat-Shamir (Non-Interactive) ---")
print(f"Message = {message.decode()}")
print(f"Commitment t = {fs['t']}, Hash-based challenge e = {fs['e']}, response s = {fs['s']}")
print(f"Verification: g^s = {fs['left']}, t * y^e = {fs['right']} -> Passed = {fs['passed']}")


=== Schnorr ZKP (Interactive) - Honest Run ===
Parameters: p=23, q=11, g=4
Secret x = 2
Public key y = g^x mod p = 16

Prover commitment t = 1 (r = 0)
Verifier challenge e = 1
Response s = 2
Verification: g^s = 16, t * y^e = 16 -> Passed = True

--- Cheating Prover Example ---
Cheating prover prepared t = 18, s = 3
Verifier challenge e = 0
Verification: g^s = 18, t * y^e = 18 -> Passed = True

--- Cheating Probability Experiment ---
Trials = 1000, Successful cheats = 481, Empirical success rate = 0.4810

--- Fiat-Shamir (Non-Interactive) ---
Message = Schnorr test message
Commitment t = 9, Hash-based challenge e = 1, response s = 10
Verification: g^s = 6, t * y^e = 6 -> Passed = True
